In [9]:
import numpy as np
import pandas as pd
import random as rd
from random import randint
import matplotlib.pyplot as plt

# Define Item Number (1, 2, .., 10)
item_number = np.arange(1,11)
print(item_number)

# Define Weight
weight = [20,25,10,5,4,12,17,17,18,19] #np.random.randint(1, 15, size = 10)
print(weight)

# Define Value
value = [100,200,150,120,450,780,1000,65,80,900] #np.random.randint(10, 750, size = 10)
print(weight)

# Define Threshold
knapsack_threshold = 100    #Maximum weight that the bag of thief can hold 

print('The list is as follows:')
print('Item No.   Weight   Value')
for i in range(item_number.shape[0]):
    print('{0}          {1}         {2}\n'.format(item_number[i], weight[i], value[i]))

[ 1  2  3  4  5  6  7  8  9 10]
[20, 25, 10, 5, 4, 12, 17, 17, 18, 19]
[20, 25, 10, 5, 4, 12, 17, 17, 18, 19]
The list is as follows:
Item No.   Weight   Value
1          20         100

2          25         200

3          10         150

4          5         120

5          4         450

6          12         780

7          17         1000

8          17         65

9          18         80

10          19         900



In [142]:
# Define the number of individuals (solutions) per population
solutions_per_pop = 8

# Define the shape of a population
pop_size = (solutions_per_pop, item_number.shape[0])
print('Population size = {}'.format(pop_size))

# Population initialization
initial_population = np.random.randint(2, size = pop_size)
initial_population = initial_population.astype(int)

# Define the number of generations
num_generations = 50
print('Initial population: \n{}'.format(initial_population))

Population size = (8, 10)
Initial population: 
[[0 1 1 1 1 1 0 1 0 1]
 [0 0 0 0 0 1 0 1 1 1]
 [1 1 0 0 1 1 0 1 1 1]
 [0 1 1 0 1 1 1 0 0 0]
 [0 0 1 0 0 1 0 0 1 0]
 [0 0 0 1 1 1 1 0 1 0]
 [1 0 1 1 1 1 0 0 1 0]
 [0 1 0 1 1 0 0 1 0 0]]


In [143]:
def cal_fitness(weight, value, population, threshold):
    fitness = np.empty(population.shape[0])
    for i in range(population.shape[0]):
        S1 = np.sum(population[i] * value)
        S2 = np.sum(population[i] * weight)
        if S2 <= threshold:
            fitness[i] = S1
        else :
            fitness[i] = 0 
    return fitness.astype(int) 

In [150]:
def weighted_random_choice(choices):
    max = sum(choices)
    pick = rd.uniform(0, max)
    current = 0
    for i,value in enumerate(choices):
        current =current+value
        if current > pick:
            return i

In [151]:
def selection(fitness, num_parents, population):
    fitness = list(fitness)
    parents = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        index=weighted_random_choice(fitness)
        parents[i,:] = population[index, :]
    return parents

In [152]:
def crossover(parents, num_offsprings):
    offsprings = np.empty((num_offsprings, parents.shape[1]))
    crossover_point = int(parents.shape[1]/2)
    crossover_rate = 0.8
    i=0
    while (parents.shape[0] < num_offsprings):
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        x = rd.random()
        if x > crossover_rate:
            continue
        parent1_index = i%parents.shape[0]
        parent2_index = (i+1)%parents.shape[0]
        offsprings[i,0:crossover_point] = parents[parent1_index,0:crossover_point]
        offsprings[i,crossover_point:] = parents[parent2_index,crossover_point:]
        i=+1
    return offsprings 

In [156]:
def mutation(offsprings):
    mutants = np.empty((offsprings.shape))
    mutation_rate = 0.4
    for i in range(mutants.shape[0]):
        random_value = rd.random()
        mutants[i,:] = offsprings[i,:]
        if random_value > mutation_rate:
            continue
        int_random_value = randint(0,offsprings.shape[1]-1)    
        if mutants[i,int_random_value] == 0 :
            mutants[i,int_random_value] = 1
        else :
            mutants[i,int_random_value] = 0
    return mutants   

In [157]:
def optimize(weight, value, population, pop_size, num_generations, threshold):
    parameters, fitness_history = [], []
    num_parents = int(pop_size[0])
    num_offsprings = pop_size[0] 
    for i in range(num_generations-1):
        fitness = cal_fitness(weight, value, population, threshold)
        fitness_history.append(fitness)
        print("generation {}".format(i))
        for j in range(pop_size[0]):
            print(population[j,:],fitness[j])
        parents = selection(fitness, num_parents, population)
        offsprings = crossover(parents, num_offsprings)
        mutants = mutation(offsprings)
        #population[0:parents.shape[0], :] = parents
        population = mutants
        
    print('Last generation: \n{}\n'.format(population)) 
    fitness_last_gen = cal_fitness(weight, value, population, threshold)      
    print('Fitness of the last generation: \n{}\n'.format(fitness_last_gen))
    max_fitness = np.where(fitness_last_gen == np.max(fitness_last_gen))
    parameters.append(population[max_fitness[0][0],:])
    return parameters, fitness_history

In [158]:
parameters, fitness_history = optimize(weight, value, initial_population, pop_size, num_generations, knapsack_threshold)
print('The optimized parameters for the given inputs are: \n{}'.format(parameters))
selected_items = item_number * parameters
print('\nSelected items that will maximize the knapsack without breaking it:')
for i in range(selected_items.shape[1]):
  if selected_items[0][i] != 0:
    {
     print('{}\n'.format(selected_items[0][i]))
     }

generation 0
[0 1 1 1 1 1 0 1 0 1] 2665
[0 0 0 0 0 1 0 1 1 1] 1825
[1 1 0 0 1 1 0 1 1 1] 0
[0 1 1 0 1 1 1 0 0 0] 2580
[0 0 1 0 0 1 0 0 1 0] 1010
[0 0 0 1 1 1 1 0 1 0] 2430
[1 0 1 1 1 1 0 0 1 0] 1680
[0 1 0 1 1 0 0 1 0 0] 835
generation 1
[1. 0. 1. 0. 1. 1. 1. 1. 0. 1.] 3445
[1. 0. 1. 0. 1. 1. 1. 1. 0. 1.] 3445
[0. 0. 1. 0. 1. 1. 0. 1. 0. 1.] 2345
[0. 0. 1. 0. 1. 1. 1. 1. 0. 1.] 3345
[0. 0. 1. 0. 1. 0. 1. 1. 0. 1.] 2565
[0. 0. 1. 1. 1. 1. 0. 1. 0. 1.] 2465
[1. 0. 1. 0. 0. 1. 0. 1. 0. 1.] 1995
[0. 0. 1. 0. 1. 1. 0. 1. 1. 1.] 2425
generation 2
[0. 1. 1. 0. 1. 1. 0. 0. 0. 0.] 1580
[0. 1. 1. 1. 1. 1. 0. 1. 0. 1.] 2665
[0. 0. 0. 1. 1. 1. 1. 0. 1. 0.] 2430
[0. 0. 1. 1. 1. 1. 0. 1. 0. 1.] 2465
[0. 0. 0. 1. 1. 1. 1. 0. 0. 0.] 2350
[0. 1. 0. 0. 1. 0. 0. 1. 0. 0.] 715
[0. 0. 0. 0. 0. 1. 0. 0. 1. 1.] 1760
[0. 1. 1. 0. 1. 1. 1. 0. 0. 0.] 2580
generation 3
[0. 0. 1. 1. 1. 1. 0. 1. 0. 1.] 2465
[1. 0. 1. 0. 1. 1. 1. 1. 0. 1.] 3445
[0. 0. 1. 0. 1. 1. 1. 1. 0. 1.] 3345
[1. 0. 0. 0. 1. 1. 1. 1. 0. 1.] 32

[0. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 2660
[0. 0. 0. 0. 1. 1. 1. 0. 1. 0.] 2310
[0. 0. 1. 0. 1. 1. 1. 1. 1. 0.] 2525
[0. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 2660
generation 43
[1. 0. 1. 1. 0. 1. 1. 1. 0. 0.] 2215
[1. 0. 1. 0. 1. 0. 0. 0. 0. 1.] 1600
[1. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3115
[1. 1. 1. 0. 0. 1. 0. 0. 0. 1.] 2130
[1. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3115
[1. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3115
[0. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3015
[1. 1. 0. 0. 1. 0. 0. 0. 0. 1.] 1650
generation 44
[0. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 2660
[0. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 2660
[0. 0. 0. 0. 1. 1. 1. 1. 1. 0.] 2375
[0. 0. 1. 0. 1. 1. 1. 1. 1. 0.] 2525
[1. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 0
[0. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 2660
[0. 1. 1. 0. 1. 1. 1. 0. 1. 0.] 2660
[1. 0. 1. 0. 1. 1. 1. 0. 1. 0.] 2560
generation 45
[1. 0. 0. 0. 1. 0. 0. 0. 0. 1.] 1450
[1. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3115
[1. 0. 1. 1. 0. 1. 1. 1. 0. 0.] 2215
[1. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3115
[1. 0. 1. 1. 0. 1. 1. 1. 0. 1.] 3115
[1. 0. 1. 0. 0. 0. 0. 0. 0. 1.] 1150